In [ ]:
%mkdir krv_tokenizer

In [ ]:
import numpy as np
import pandas as pd
import torch 
import random
import pickle
import transformers

from transformers import RobertaTokenizerFast
from transformers import pipeline

from sin_tokenizer import SinTokenizer
from transformers_ts.roberta_ml_trainer import prepare_trainer

In [ ]:
from_pretrained = None
n = 5
num_epochs_iter = 1
num_eval_samples = 1

In [ ]:
tok = SinTokenizer(delta = 0.01, model_max_length = 40)
tok.make_data_lines()

In [ ]:
for i in range(num_epochs_iter):
  trainer = prepare_trainer(len(tok.get_vocab()), tok, num_train_epochs=5, from_pretrained = from_pretrained)
  trainer.train()

  fill_mask = pipeline(
      "fill-mask",
      model=trainer.model,
      tokenizer=tok.tokenizer,
      device = 0
  )

  rmse_cum = 0
  rmse_vals = []
  for k in range(num_eval_samples):
    masked_string, target_string, vals = tok.make_masked_data(n)
    result = fill_mask(masked_string)
    preds = [result[0][0]['token_str'], result[1][0]['token_str'], result[2][0]['token_str'], result[3][0]['token_str'], result[4][0]['token_str']]
    preds = tok.convert_tokens_to_vals(preds)
    rmse = np.sqrt(np.mean((preds-vals)**2))
    rmse_cum += rmse
    rmse_vals.append(rmse)

  print(f'epoch iter = {i}, rmse = {rmse_cum/(n*num_eval_samples)}, std = {np.std(rmse_vals)}')


  data_path = 'pretrained/'
  from_pretrained = data_path + "krvRoberta"

  trainer.save_model(from_pretrained)
  with open(data_path + 'tok.pickle', 'wb') as f:
    pickle.dump(tok, f)

In [ ]:
masked_string, target_string, vals = tok.make_masked_data(n)
masked_indexes = [i for i,v in enumerate(list(np.reshape(masked_string.split(), -1))) if v == '<mask>']
target = [t for i, t in enumerate(list(np.reshape(target_string.split(), -1))) if i in masked_indexes]

result = fill_mask(masked_string)
true_vals = tok.convert_tokens_to_vals(target)
preds = [result[i][0]['token_str'] for i in range(np.shape(result)[1])]
pred_vals = tok.convert_tokens_to_vals(preds)
rmse = np.sqrt(np.mean([(x - y)**2 for x,y in zip(pred_vals, true_vals)]))
rmse